In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

def seasonProgression(dfR):
    totalGames = len(dfR)
    curWeek = int(dfR['week']) + 1
    return curWeek/(2*totalGames)

def winningTeam(dfR):
    if dfR['home_points'] > dfR['away_points']:
        return dfR['home_team']
    else:
        return dfR['away_team']
    
def teamWin(dfR, team):
    if dfR.winner == team:
        return True
    else:
        return False
    
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference
    
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.
    
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points
    
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15
    
def winQ(dfR,week): 
    if dfR.opp_conference == None:
        oppWinPct = 0.5
    else:
#         print(dfR.opp_conference)
        oppWinPct = FBSdict[dfR.opp].loc[FBSdict[dfR.opp].week <= week, 'winPct'].tail(1).item()
    
    if dfR.teamWin == True:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * dfR.seasonProg * dfR.conf_mult * winMult*(dfR.loc_mult + 0.01*dfR.point_diff)

def calcPct(team, week):
    winqweek = 'win_q_'+str(week)
    wins = FBSdict[team].loc[FBSdict[team].week <= week, 'wins'].tail(1).item()
    losses = FBSdict[team].loc[FBSdict[team].week <= week, 'losses'].tail(1).item()
    return 0.9 * FBSdict[team][winqweek].mean() + 0.01*(wins-losses)

In [3]:
jsonFile = '2019/2019schedule.json'

In [4]:
curSch = pd.read_json(jsonFile)

In [5]:
curSch.loc[curSch.away_team == 'Notre Dame','away_conference'] = 'Notre Dame'
curSch.loc[curSch.home_team == 'Notre Dame','home_conference'] = 'Notre Dame'

In [6]:
FBSteams = curSch['home_team'].unique()
FBSteams.sort()

In [7]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = curSch[(curSch['away_team'] == team) | 
                           (curSch['home_team'] == team)]
    FBSdict[team] = FBSdict[team][np.isfinite(FBSdict[team].away_points)]
    FBSdict[team] = FBSdict[team].reset_index()

In [8]:
FBSbar = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar.set_description(key)
    FBSdict[key]['seasonProg'] = FBSdict[key].apply(lambda row: seasonProgression(row),axis = 1)
    FBSdict[key]['winner'] = FBSdict[key].apply(lambda row: winningTeam(row),axis = 1)
    FBSdict[key]['teamWin'] = FBSdict[key].apply(lambda row: teamWin(row,key),axis = 1)
    
    wins = 0
    losses = 0
    winList = []
    lossList = []
    for row in FBSdict[key].itertuples():
    #     print(row.winner)
        if row.teamWin:
            wins += 1
        else:
            losses += 1
        winList.append(wins)
        lossList.append(losses)
        
    FBSdict[key]['wins'] = winList
    FBSdict[key]['losses'] = lossList
    
    hometeam = FBSdict[key]['home_team'] == key
    FBSdict[key]['home'] = hometeam
    
    FBSdict[key]['winPct'] = FBSdict[key]['wins']/(FBSdict[key]['losses'] + FBSdict[key]['wins'])
    FBSdict[key]['opp'] = FBSdict[key].apply(lambda row: findOpp(row), axis = 1)
    FBSdict[key]['opp_conference'] = FBSdict[key].apply(lambda row: findOppConf(row), axis = 1)
    FBSdict[key]['point_diff'] = FBSdict[key].apply(lambda row: pointDiff(row,key), axis = 1)
    FBSdict[key]['loc_mult'] = FBSdict[key].apply(lambda row: locMult(row), axis = 1)
    FBSdict[key]['conf_mult'] = FBSdict[key].apply(lambda row: confMult(row), axis = 1)
    
    FBSbar.update(1)

In [9]:
FBSbar2 = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar2.set_description(key)
    for w in range(4,16):
        FBSdict[key]['win_q_'+str(w)] = FBSdict[key].apply(lambda row: winQ(row, w),axis=1)
    FBSbar2.update()

In [10]:
FBSbar3 = tqdm(total=len(FBSteams))
FBSpct = {}
for key in FBSdict:
    FBSbar3.set_description(key)
    pctList = []
    for w in range(4,16):
        pctList.append(calcPct(key,w))
    FBSpct[key] = pctList
    FBSbar3.update()

In [11]:
PCTdf = pd.DataFrame.from_dict(FBSpct,orient='index',columns=range(4,16))

In [12]:
PCTnormdf = pd.DataFrame(index = FBSteams,columns=range(4,16))

for w in range(4,16):
    x = PCTdf[w].values.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    xScaled = min_max_scaler.fit_transform(x)
    PCTnormdf[w] = xScaled.reshape(130)

PCTnormdf.to_csv('2019/PCT.csv')

In [13]:
Ranks = PCTdf.rank(method='first',ascending=False).astype('int64')
Ranks.to_csv('2019/Ranks.csv')

In [14]:
for w in range(4,16):
    Wdf = pd.concat([PCTnormdf[w],Ranks[w]],axis=1)
    Wdf.columns = ['PCT','Rank']
    Wdf = Wdf.sort_values('Rank')
    Wdf.to_csv('2019/W%s.csv'%w)